https://zenn.dev/umi_mori/books/prompt-engineer/viewer/langchain_prompt

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain import PromptTemplate
from langchain.llms import OpenAI

template = """
{subject}について30文字で教えて。
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["subject"]
)
prompt_text = prompt.format(subject="ITエンジニア")
print(prompt_text)


ITエンジニアについて30文字で教えて。



In [3]:
llm = OpenAI(model_name="text-davinci-003")
print(llm(prompt_text))


ITエンジニアは、ソフトウェアおよびハードウェアを構築し、保守する専門家です。


In [4]:
from langchain import PromptTemplate
from langchain import FewShotPromptTemplate
from langchain.llms import OpenAI

examples = [
    {"fruit": "りんご", "color": "赤"},
    {"fruit": "キウイ", "color": "緑"},
    {"fruit": "ぶどう", "color": "紫"},
]

example_formatter_template = """
フルーツ: {fruit}
色: {color}\n
"""
example_prompt = PromptTemplate(
    template=example_formatter_template,
    input_variables=["fruit", "color"]
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="フルーツの色を漢字で教えて。",
    suffix="フルーツ: {input}\n色:",
    input_variables=["input"],
    example_separator="\n\n",

)

prompt_text = few_shot_prompt.format(input="オレンジ")
print(prompt_text)

llm = OpenAI(model_name="text-davinci-003")
print(llm(prompt_text))

フルーツの色を漢字で教えて。


フルーツ: りんご
色: 赤




フルーツ: キウイ
色: 緑




フルーツ: ぶどう
色: 紫



フルーツ: オレンジ
色:
 橙


In [5]:
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chat_models import ChatOpenAI

system_template="あなたは、質問者からの質問を{language}で回答するAIです。"
human_template="質問者：{question}"
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
prompt_message_list = chat_prompt.format_prompt(language="日本語", question="ITエンジニアについて30文字で教えて。").to_messages()

print(prompt_message_list)

chat = ChatOpenAI(model_name="gpt-3.5-turbo")
chat(prompt_message_list)

[SystemMessage(content='あなたは、質問者からの質問を日本語で回答するAIです。', additional_kwargs={}), HumanMessage(content='質問者：ITエンジニアについて30文字で教えて。', additional_kwargs={}, example=False)]


AIMessage(content='技術的な知識と問題解決能力を持つコンピューターシステムの開発・運用に従事する職業。', additional_kwargs={}, example=False)

In [6]:
from langchain.prompts import load_prompt

prompt = load_prompt("lc://prompts/conversation/prompt.json")
prompt_text = prompt.format(history="こんにちは、ますみです。", input="ITエンジニアについて30文字で教えて。")
print(prompt_text)

No `_type` key found, defaulting to `prompt`.


The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
こんにちは、ますみです。
Human: ITエンジニアについて30文字で教えて。
AI:


In [9]:
from typing import List, Dict
from langchain.prompts.example_selector.base import BaseExampleSelector
import numpy as np

class CustomExampleSelector(BaseExampleSelector):

    def __init__(self, examples: List[Dict[str, str]]):
        self.examples = examples

    def add_example(self, example: Dict[str, str]) -> None:
        """新しい教師データを格納するめの関数"""
        self.examples.append(example)

    def select_examples(self, input_variables: Dict[str, str]) -> List[dict]:
        """教師データを選択するための関数"""
        return np.random.choice(self.examples, size=2, replace=False)

In [10]:
examples = [
    {"fruit": "りんご", "color": "赤"},
    {"fruit": "キウイ", "color": "緑"},
    {"fruit": "ぶどう", "color": "紫"},
]

# 教師データの登録
example_selector = CustomExampleSelector(examples)

# 教師データの出力
print(example_selector.examples)

[{'fruit': 'りんご', 'color': '赤'}, {'fruit': 'キウイ', 'color': '緑'}, {'fruit': 'ぶどう', 'color': '紫'}]


In [11]:
# 教師データの追加
example_selector.add_example({"fruit": "オレンジ", "color": "橙"})
print(example_selector.examples)

[{'fruit': 'りんご', 'color': '赤'}, {'fruit': 'キウイ', 'color': '緑'}, {'fruit': 'ぶどう', 'color': '紫'}, {'fruit': 'オレンジ', 'color': '橙'}]


In [15]:
# 教師データの選択
print(example_selector.select_examples({}))

[{'fruit': 'りんご', 'color': '赤'} {'fruit': 'ぶどう', 'color': '紫'}]


In [16]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)

llm = OpenAI(model_name="text-davinci-003")
_input = prompt.format(subject="Programming Language")
output = llm(_input)
output_parser.parse(output)

['Java', 'C++', 'Python', 'JavaScript', 'Ruby']

In [17]:
from typing import List
from pydantic import BaseModel, Field

from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.output_parsers import PydanticOutputParser

class Job(BaseModel):
    name: str = Field(description="Name of the job")
    skill_list: List[str] = Field(description="List of skills required for that job")

parser = PydanticOutputParser(pydantic_object=Job)
prompt = PromptTemplate(
    template="{query}\n\n{format_instructions}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)
_input = prompt.format_prompt(query="Tell me the skills required for frontend engineer.")
print(_input)

llm = OpenAI(model_name="text-davinci-003")
output = llm(_input.to_string())
print(output)

parser.parse(output)

text='Tell me the skills required for frontend engineer.\n\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"title": "Name", "description": "Name of the job", "type": "string"}, "skill_list": {"title": "Skill List", "description": "List of skills required for that job", "type": "array", "items": {"type": "string"}}}, "required": ["name", "skill_list"]}\n```\n'

And the output instance:
```
{"name": "Frontend Engineer", "skill_list": ["HTML", "CSS", "JavaScript", "jQuery", "Responsive Design", "Cross-browser Compatibility", "Version Control", "Testing and Deb

Job(name='Frontend Engineer', skill_list=['HTML', 'CSS', 'JavaScript', 'jQuery', 'Responsive Design', 'Cross-browser Compatibility', 'Version Control', 'Testing and Debugging'])

In [20]:
from typing import List

from pydantic import BaseModel, Field

from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import (
    PydanticOutputParser,
    OutputFixingParser,
)

class Job(BaseModel):
    name: str = Field(description="Name of the job")
    skill_list: List[str] = Field(description="List of skills required for that job")

misformatted = "{'skills': ['HTML', 'CSS', 'JS', 'TS'], 'job': 'Frontend Engineer'}"

parser = PydanticOutputParser(pydantic_object=Job)

new_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI(model_name="gpt-3.5-turbo"))
new_parser.parse(misformatted)

Job(name='Frontend Engineer', skill_list=['HTML', 'CSS', 'JS', 'TS'])